In [ ]:
'''
IKCEST 3rd: object detection + segmentation
1.This is a reference notebook for generating json file in the required format for the last three classes
(corsswalk, solid lane line, dotted lane line)

2.Here I use the open-mmlab mmdetection framework(It's fine if you use paddle or detectron).
my version: 
python: 3.7.10
torch: 1.7.1+cu110
torchvision: 0.8.2+cu110
mmcv-full: 1.3.1
mmdet: 2.11.0

3. What model do I use and why?
mask-rcnn, because it's quite easy to generate polygon masks in the required format

4. I have upload my train, val, annotations data, also, I uploaded my config python script and trained weights.

5. Eh... it take a long time to generate result jsonfile, whick is no doubt exceeding the time limit. (20fps)

6. If you want to use this notebook, make sure you have installed mmdetection, and verify your installation.
If every thing is fine, then you can start!
'''

In [1]:
from mmdet.apis import init_detector, inference_detector, show_result_pyplot
import mmcv
import cv2
import numpy as np
import json
import time

In [2]:
config_file = '../configs/mask_rcnn_for_segm.py' # load config file, put the config file you use in config folder
checkpoint_file = '../work_dirs/mask_rcnn_for_segm/latest.pth' # load weights, your trained weights (*.pth)

In [3]:
model = init_detector(config_file, checkpoint_file, device='cuda:0') # prepare the model 

Use load_from_local loader


In [4]:
import os
imgs_list = os.listdir('/home/zyn/Desktop/IKCEST3rd/test_dataset/test_dataset/')# the test img folder
assert len(imgs_list) == 2000
imgs_list.sort()

In [5]:
imgs = ['/home/zyn/Desktop/IKCEST3rd/test_dataset/test_dataset/'+img for img in imgs_list]

In [6]:
# note: caculate poly area , this function 
def calculate_poly(binary_mask):
    '''
    input: binary mask ,shape is (height, width)
    return: poly area coordinates [x1, y1, x2, y2, x3, y3, ...]
    note: we want poly contour coordinates, instead of area coordinates !!!
    
    '''
    assert len(binary_mask.shape) == 2
    y, x = np.where(binary_mask == True)
    x_float = x.astype(float)
    y_float = y.astype(float)
    
    coord = list(zip(list(x_float), list(y_float)))
    poly = [y for x in coord for y in x]
    
#     print(x_float)
#     print(y_float)
#     print(poly)
    
    return poly  

def get_contour(img):
    '''
    input: binary mask
    return: img, contours
    '''
#     img_gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    ret, img_bin = cv2.threshold(img, 127, 255, cv2.THRESH_BINARY)
    contours, hierarchy = cv2.findContours(img_bin, cv2.RETR_LIST, cv2.CHAIN_APPROX_SIMPLE)
    return img, contours

def calculate_poly_contour(binary_mask):
    '''
    input: binary mask
    return: poly contour coordinates [x1, y1, x2, y2, x3, y3, ...]
    
    '''
    assert len(binary_mask.shape) == 2 # binary shape is (h, w)
    img_src = binary_mask.astype(int) # change binary mask from bool to int (0, 1)
    img_src[img_src != 0] = 255 # for find contour, make "1" equals to 155
    img = np.array(img_src, dtype= 'uint8') # change img dtype to uint8 for find contour
    
    img_gray, contour = get_contour(img) # get contour
    mask_out_edge = np.zeros(img_gray.shape, np.uint8) 
    cv2.drawContours(mask_out_edge, contour, 0, 255, 2) # draw contour on black background img

#     cv2.imwrite("break333.jpg", mask_out_edge)
    assert mask_out_edge.dtype == 'uint8'
    y, x = np.where(mask_out_edge == 255) # find contour coordinates
    x_float = x.astype(float) # change type from int to float
    y_float = y.astype(float) # change type from int to float
    
    coord = list(zip(list(x_float), list(y_float)))
    poly = [y for x in coord for y in x]
    
#     print(x_float)
#     print(y_float)
#     print(len(poly))
    
    return poly  
    

In [7]:
def per_img_polymask(result_per_img: list, file_name: str, file_path:str):
    '''
    build a dict for each instance
    inputs: 
        result_per_img: mask predict result, list
        file_name: img name
        file_path: img path
    
    return: a list of instance pred dict (how many instances in this img)
    
    '''
    per_img_output = []
    img = cv2.imread(file_path)
#     per_instance_output = {}
    class_type = [8, 9, 10] # represents solid lane, dotted lane, crosswalk
    per_img_masks_numper = (len(result_per_img[1][0]) + len(result_per_img[1][1]) + len(result_per_img[1][2]))
    solid_lane_ins_number = len(result_per_img[1][0])    # number of solid lane instances in this img
    dotted_lane_ins_number = len(result_per_img[1][1])   # number of dotted lane instances in this img
    crosswalk_ins_number = len(result_per_img[1][2])     # number of crosswalk instances in this img
    
    solid_lane_instances = result_per_img[1][0]
    dotted_lane_instances = result_per_img[1][1]
    crosswalk_instances = result_per_img[1][2]
    
    
    image_id = int(file_name.split('.')[0]) # got image_id for this picture
    img_height = img.shape[0]
    img_width = img.shape[1]
    
    ####################### solid lane #######################
    if solid_lane_instances:
        for item in solid_lane_instances: # deal with each solid instance in this img
            assert item.shape == (img_height, img_width)
            solid_obj = {}
            solid_obj['image_id'] = image_id
            solid_obj['type'] = class_type[0]
            solid_obj['x'] = -1
            solid_obj['y'] = -1
            solid_obj['width'] = -1
            solid_obj['height'] = -1
            solid_obj['segmentation'] = calculate_poly_contour(item)
            per_img_output.append(solid_obj)
    
    ####################### dotted lane #######################
    if dotted_lane_instances:
        for item in dotted_lane_instances: # deal with each dotted instance in this img
            assert item.shape == (img_height, img_width)
            dotted_obj = {}
            dotted_obj['image_id'] = image_id
            dotted_obj['type'] = class_type[1]
            dotted_obj['x'] = -1
            dotted_obj['y'] = -1
            dotted_obj['width'] = -1
            dotted_obj['height'] = -1
            dotted_obj['segmentation'] = calculate_poly_contour(item)
            per_img_output.append(dotted_obj)

    ####################### crosswalk #######################
    if crosswalk_instances:
        for item in crosswalk_instances: # deal with each crosswalk instance in this img
            assert item.shape == (img_height, img_width)
            crosswalk_obj = {}
            crosswalk_obj['image_id'] = image_id
            crosswalk_obj['type'] = class_type[2]
            crosswalk_obj['x'] = -1
            crosswalk_obj['y'] = -1
            crosswalk_obj['width'] = -1
            crosswalk_obj['height'] = -1
            crosswalk_obj['segmentation'] = calculate_poly_contour(item)
            per_img_output.append(crosswalk_obj)
    
    return per_img_output   
    
    

In [8]:
#img = '07903.jpg'
result_list = []
count = 0
t1 = time.time()
for i, item in enumerate(imgs): # deal with each img in test dataset
    result_per_img = inference_detector(model, item)
    print(f'path is: {item}')
    print(f'name is: {imgs_list[i]}')
    assert len(result_per_img) == 2
    assert len(result_per_img[1]) == 3 # There are 3 classes for segm, from 8-10 :solid, dotted, crosswalk
    
    per_img_anno = per_img_polymask(result_per_img, imgs_list[i], item)
    
    result_list.extend(per_img_anno)
    print(f'{len(per_img_anno)} instances in img {imgs_list[i]}')
    print(f'img {imgs_list[i]} has been written in result files!')
    print()

t2 = time.time()
print(f'time cost: {(t2-t1):.3f}')




/home/zyn/mmdetection/mmdet/datasets/utils.py:68: UserWarning: "ImageToTensor" pipeline is replaced by "DefaultFormatBundle" for batch inference. It is recommended to manually replace it in the test data pipeline in your config file.
  'data pipeline in your config file.', UserWarning)


path is: /home/zyn/Desktop/IKCEST3rd/test_dataset/test_dataset/08001.jpg
name is: 08001.jpg
7 instances in img 08001.jpg
img 08001.jpg has been written in result files!

path is: /home/zyn/Desktop/IKCEST3rd/test_dataset/test_dataset/08002.jpg
name is: 08002.jpg
1 instances in img 08002.jpg
img 08002.jpg has been written in result files!

path is: /home/zyn/Desktop/IKCEST3rd/test_dataset/test_dataset/08003.jpg
name is: 08003.jpg
12 instances in img 08003.jpg
img 08003.jpg has been written in result files!

path is: /home/zyn/Desktop/IKCEST3rd/test_dataset/test_dataset/08004.jpg
name is: 08004.jpg
37 instances in img 08004.jpg
img 08004.jpg has been written in result files!

path is: /home/zyn/Desktop/IKCEST3rd/test_dataset/test_dataset/08005.jpg
name is: 08005.jpg
24 instances in img 08005.jpg
img 08005.jpg has been written in result files!

path is: /home/zyn/Desktop/IKCEST3rd/test_dataset/test_dataset/08006.jpg
name is: 08006.jpg
9 instances in img 08006.jpg
img 08006.jpg has been wri

12 instances in img 08049.jpg
img 08049.jpg has been written in result files!

path is: /home/zyn/Desktop/IKCEST3rd/test_dataset/test_dataset/08050.jpg
name is: 08050.jpg
33 instances in img 08050.jpg
img 08050.jpg has been written in result files!

path is: /home/zyn/Desktop/IKCEST3rd/test_dataset/test_dataset/08051.jpg
name is: 08051.jpg
8 instances in img 08051.jpg
img 08051.jpg has been written in result files!

path is: /home/zyn/Desktop/IKCEST3rd/test_dataset/test_dataset/08052.jpg
name is: 08052.jpg
19 instances in img 08052.jpg
img 08052.jpg has been written in result files!

path is: /home/zyn/Desktop/IKCEST3rd/test_dataset/test_dataset/08053.jpg
name is: 08053.jpg
17 instances in img 08053.jpg
img 08053.jpg has been written in result files!

path is: /home/zyn/Desktop/IKCEST3rd/test_dataset/test_dataset/08054.jpg
name is: 08054.jpg
19 instances in img 08054.jpg
img 08054.jpg has been written in result files!

path is: /home/zyn/Desktop/IKCEST3rd/test_dataset/test_dataset/0805

path is: /home/zyn/Desktop/IKCEST3rd/test_dataset/test_dataset/08099.jpg
name is: 08099.jpg
27 instances in img 08099.jpg
img 08099.jpg has been written in result files!

path is: /home/zyn/Desktop/IKCEST3rd/test_dataset/test_dataset/08100.jpg
name is: 08100.jpg
17 instances in img 08100.jpg
img 08100.jpg has been written in result files!

path is: /home/zyn/Desktop/IKCEST3rd/test_dataset/test_dataset/08101.jpg
name is: 08101.jpg
0 instances in img 08101.jpg
img 08101.jpg has been written in result files!

path is: /home/zyn/Desktop/IKCEST3rd/test_dataset/test_dataset/08102.jpg
name is: 08102.jpg
5 instances in img 08102.jpg
img 08102.jpg has been written in result files!

path is: /home/zyn/Desktop/IKCEST3rd/test_dataset/test_dataset/08103.jpg
name is: 08103.jpg
17 instances in img 08103.jpg
img 08103.jpg has been written in result files!

path is: /home/zyn/Desktop/IKCEST3rd/test_dataset/test_dataset/08104.jpg
name is: 08104.jpg
17 instances in img 08104.jpg
img 08104.jpg has been wr

path is: /home/zyn/Desktop/IKCEST3rd/test_dataset/test_dataset/08147.jpg
name is: 08147.jpg
8 instances in img 08147.jpg
img 08147.jpg has been written in result files!

path is: /home/zyn/Desktop/IKCEST3rd/test_dataset/test_dataset/08148.jpg
name is: 08148.jpg
4 instances in img 08148.jpg
img 08148.jpg has been written in result files!

path is: /home/zyn/Desktop/IKCEST3rd/test_dataset/test_dataset/08149.jpg
name is: 08149.jpg
8 instances in img 08149.jpg
img 08149.jpg has been written in result files!

path is: /home/zyn/Desktop/IKCEST3rd/test_dataset/test_dataset/08150.jpg
name is: 08150.jpg
14 instances in img 08150.jpg
img 08150.jpg has been written in result files!

path is: /home/zyn/Desktop/IKCEST3rd/test_dataset/test_dataset/08151.jpg
name is: 08151.jpg
13 instances in img 08151.jpg
img 08151.jpg has been written in result files!

path is: /home/zyn/Desktop/IKCEST3rd/test_dataset/test_dataset/08152.jpg
name is: 08152.jpg
8 instances in img 08152.jpg
img 08152.jpg has been writ

...

23 instances in img 09763.jpg
img 09763.jpg has been written in result files!

path is: /home/zyn/Desktop/IKCEST3rd/test_dataset/test_dataset/09764.jpg
name is: 09764.jpg
19 instances in img 09764.jpg
img 09764.jpg has been written in result files!

path is: /home/zyn/Desktop/IKCEST3rd/test_dataset/test_dataset/09765.jpg
name is: 09765.jpg
14 instances in img 09765.jpg
img 09765.jpg has been written in result files!

path is: /home/zyn/Desktop/IKCEST3rd/test_dataset/test_dataset/09766.jpg
name is: 09766.jpg
12 instances in img 09766.jpg
img 09766.jpg has been written in result files!

path is: /home/zyn/Desktop/IKCEST3rd/test_dataset/test_dataset/09767.jpg
name is: 09767.jpg
13 instances in img 09767.jpg
img 09767.jpg has been written in result files!

path is: /home/zyn/Desktop/IKCEST3rd/test_dataset/test_dataset/09768.jpg
name is: 09768.jpg
1 instances in img 09768.jpg
img 09768.jpg has been written in result files!

path is: /home/zyn/Desktop/IKCEST3rd/test_dataset/test_dataset/0976

12 instances in img 09811.jpg
img 09811.jpg has been written in result files!

path is: /home/zyn/Desktop/IKCEST3rd/test_dataset/test_dataset/09812.jpg
name is: 09812.jpg
5 instances in img 09812.jpg
img 09812.jpg has been written in result files!

path is: /home/zyn/Desktop/IKCEST3rd/test_dataset/test_dataset/09813.jpg
name is: 09813.jpg
15 instances in img 09813.jpg
img 09813.jpg has been written in result files!

path is: /home/zyn/Desktop/IKCEST3rd/test_dataset/test_dataset/09814.jpg
name is: 09814.jpg
8 instances in img 09814.jpg
img 09814.jpg has been written in result files!

path is: /home/zyn/Desktop/IKCEST3rd/test_dataset/test_dataset/09815.jpg
name is: 09815.jpg
12 instances in img 09815.jpg
img 09815.jpg has been written in result files!

path is: /home/zyn/Desktop/IKCEST3rd/test_dataset/test_dataset/09816.jpg
name is: 09816.jpg
15 instances in img 09816.jpg
img 09816.jpg has been written in result files!

path is: /home/zyn/Desktop/IKCEST3rd/test_dataset/test_dataset/09817

16 instances in img 09860.jpg
img 09860.jpg has been written in result files!

path is: /home/zyn/Desktop/IKCEST3rd/test_dataset/test_dataset/09861.jpg
name is: 09861.jpg
15 instances in img 09861.jpg
img 09861.jpg has been written in result files!

path is: /home/zyn/Desktop/IKCEST3rd/test_dataset/test_dataset/09862.jpg
name is: 09862.jpg
4 instances in img 09862.jpg
img 09862.jpg has been written in result files!

path is: /home/zyn/Desktop/IKCEST3rd/test_dataset/test_dataset/09863.jpg
name is: 09863.jpg
26 instances in img 09863.jpg
img 09863.jpg has been written in result files!

path is: /home/zyn/Desktop/IKCEST3rd/test_dataset/test_dataset/09864.jpg
name is: 09864.jpg
12 instances in img 09864.jpg
img 09864.jpg has been written in result files!

path is: /home/zyn/Desktop/IKCEST3rd/test_dataset/test_dataset/09865.jpg
name is: 09865.jpg
15 instances in img 09865.jpg
img 09865.jpg has been written in result files!

path is: /home/zyn/Desktop/IKCEST3rd/test_dataset/test_dataset/0986

path is: /home/zyn/Desktop/IKCEST3rd/test_dataset/test_dataset/09909.jpg
name is: 09909.jpg
0 instances in img 09909.jpg
img 09909.jpg has been written in result files!

path is: /home/zyn/Desktop/IKCEST3rd/test_dataset/test_dataset/09910.jpg
name is: 09910.jpg
27 instances in img 09910.jpg
img 09910.jpg has been written in result files!

path is: /home/zyn/Desktop/IKCEST3rd/test_dataset/test_dataset/09911.jpg
name is: 09911.jpg
20 instances in img 09911.jpg
img 09911.jpg has been written in result files!

path is: /home/zyn/Desktop/IKCEST3rd/test_dataset/test_dataset/09912.jpg
name is: 09912.jpg
6 instances in img 09912.jpg
img 09912.jpg has been written in result files!

path is: /home/zyn/Desktop/IKCEST3rd/test_dataset/test_dataset/09913.jpg
name is: 09913.jpg
4 instances in img 09913.jpg
img 09913.jpg has been written in result files!

path is: /home/zyn/Desktop/IKCEST3rd/test_dataset/test_dataset/09914.jpg
name is: 09914.jpg
8 instances in img 09914.jpg
img 09914.jpg has been writ

23 instances in img 09957.jpg
img 09957.jpg has been written in result files!

path is: /home/zyn/Desktop/IKCEST3rd/test_dataset/test_dataset/09958.jpg
name is: 09958.jpg
15 instances in img 09958.jpg
img 09958.jpg has been written in result files!

path is: /home/zyn/Desktop/IKCEST3rd/test_dataset/test_dataset/09959.jpg
name is: 09959.jpg
2 instances in img 09959.jpg
img 09959.jpg has been written in result files!

path is: /home/zyn/Desktop/IKCEST3rd/test_dataset/test_dataset/09960.jpg
name is: 09960.jpg
28 instances in img 09960.jpg
img 09960.jpg has been written in result files!

path is: /home/zyn/Desktop/IKCEST3rd/test_dataset/test_dataset/09961.jpg
name is: 09961.jpg
4 instances in img 09961.jpg
img 09961.jpg has been written in result files!

path is: /home/zyn/Desktop/IKCEST3rd/test_dataset/test_dataset/09962.jpg
name is: 09962.jpg
17 instances in img 09962.jpg
img 09962.jpg has been written in result files!

path is: /home/zyn/Desktop/IKCEST3rd/test_dataset/test_dataset/09963

In [9]:
result = dict()
result['result'] = result_list

In [10]:
############# write json file for result (last 3 classes) ################
json_str = json.dumps(result)
with open('ikcest_pred_result_last_3_classes.json', 'w') as json_fff:
    json_fff.write(json_str)

print('finished!!!')



finished!!!
